#### QA over the PDF file Data

We will create a Q and A app . that can answer the questions about the PDF filed

In [1]:
import os
from dotenv import load_dotenv, load_dotenv
_= load_dotenv(load_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [3]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "data\stats.pdf"
loader = PyPDFLoader(file_path)


In [4]:

docs = loader.load()

In [5]:
print((docs))

[Document(metadata={'producer': 'Skia/PDF m120 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Report - Importance and the use of correlation in Statistics', 'source': 'data\\stats.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='ImportanceandtheuseofcorrelationinStatistics\nIntroduction\nCorrelationisastatistical measurethat expressestheextent towhichtwovariablesarelinearlyrelated. It isacommontool fordescribingsimplerelationshipswithout makingastatement about causeandeffect.Correlationcoefficientsrangefrom-1to1, withavalueof 0indicatingnolinearrelationshipbetweenthetwovariables, avalueof 1indicatingaperfect positivelinearrelationship, andavalueof -1indicatingaperfectnegativelinearrelationship.\nCorrelationisimportantinstatisticsbecauseitcanbeusedto\n1. Identifyrelationshipsbetweenvariables:Correlationcanbeusedtoidentifywhetherthereisarelationshipbetweentwovariables, andif so, whethertherelationshipispositiveornegative. Thisinformationcanbe

#### RAC: Here w can use the RAG. because of customer data

In [6]:
pip install langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_chroma import Chroma

In [8]:
!pip install langchain-huggingface sentence-transformers

In [9]:

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)

splits = text_splitter.split_documents(docs)

vector_store = Chroma.from_documents(documents = splits, embedding=embeddings)

In [11]:
retriever =  vector_store.as_retriever()

##### we can use 2 predefined chanins for creating to final chains

In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate

In [13]:
system_prompt = (
    
    "You are an assistant for question answering tasks. "
    "use the following pieces of retrieved context to answer"
    "The question. If you don't know the answer, say that you "
    " don't know. Use three sentenses maximum and keep the answer concise."
    "{context}"
    
)




In [14]:
prompt = ChatPromptTemplate.from_messages(
    
    
    [
        ("system", system_prompt),
        ("human", "{input}"),      
        
    ]
)





In [31]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain =create_retrieval_chain(retriever,question_answer_chain)
results = rag_chain.invoke({"input": "What is Correlation?"})

In [32]:
results['answer']

'Correlation is a statistical measure that expresses how two variables vary together, indicating the strength and direction of a linear relationship. Its coefficient ranges from –1 (perfect negative linear relationship) to +1 (perfect positive linear relationship), with 0 meaning no linear relationship. The value is used to identify relationships, make predictions, and develop causal models in many fields.'